In [8]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Thay bằng đường dẫn đến file JSON
SERVICE_ACCOUNT_FILE = '../../env/gen-lang-client-0447346298-03c5c4003642.json'

# Scope để truy cập Google Sheets
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# Tạo credentials
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Tạo service object
service = build('sheets', 'v4', credentials=creds)

# ID và tên sheet bạn muốn đọc
SPREADSHEET_ID = '19zPaWbW-VuYqhq1ur8vcPoV4EbDKKVDDN84vi4yuOJU'  # Thay bằng ID thật
RANGE_NAME = 'report1!A1:D10'  # Tùy theo vùng dữ liệu bạn muốn lấy

# Gọi API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=RANGE_NAME).execute()
values = result.get('values', [])

# In ra
if not values:
    print('Không có dữ liệu.')
else:
    for row in values:
        print(row)


['Date', 'Inprogress', 'Blocker', 'Completed']
['12/06/2025', 'Nghiên cứu finetuning_model\n- LoRA\n- QLoRA\n', '', '- Time optimization for analyis intent']


In [11]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

creds = service_account.Credentials.from_service_account_file(
    "../../env/gen-lang-client-0447346298-03c5c4003642.json",
    scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"]
)

service = build('sheets', 'v4', credentials=creds)


In [16]:
service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()

{'range': 'report1!A1:D10',
 'majorDimension': 'ROWS',
 'values': [['Date', 'Inprogress', 'Blocker', 'Completed'],
  ['12/06/2025',
   'Nghiên cứu finetuning_model\n- LoRA\n- QLoRA\n',
   '',
   '- Time optimization for analyis intent']]}

In [30]:
import polars as pl
import requests
from io import BytesIO

sheet_id = "19zPaWbW-VuYqhq1ur8vcPoV4EbDKKVDDN84vi4yuOJU"
csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"

# Tải dữ liệu dưới dạng bytes
response = requests.get(csv_url)
response.raise_for_status()

# Đọc CSV với encoding UTF-8
df = pl.read_csv(BytesIO(response.content), encoding="utf8")

df.to_struct()[0]


{'Date': '12/06/2025',
 'Inprogress': 'Nghiên cứu finetuning_model\n- LoRA\n- QLoRA\n',
 'Blocker': None,
 'Completed': '- Time optimization for analyis intent'}

In [75]:
url = "https://docs.google.com/spreadsheets/d/19zPaWbW-VuYqhq1ur8vcPoV4EbDKKVDDN84vi4yuOJU/edit?usp=sharing"
def get_information_from_url(url: str) -> dict:
    """
    Lấy dữ liệu báo cáo hàng ngày từ Google Sheet bằng URL
    """
    # Tải dữ liệu dưới dạng bytes
    sheet_id = url.split("/")[-2]
    csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"

    response = requests.get(csv_url)
    response.raise_for_status()

    # Đọc CSV với encoding UTF-8
    try:
        df = pl.read_csv(BytesIO(response.content), encoding="utf8")
        df_formated = df.with_columns(pl.col('Date').str.to_datetime().dt.date().alias('date_formated')).sort(by='date_formated',descending=True) # type: ignore
        for content in df_formated.iter_rows():
            if content[1]:
                return df.filter(pl.col('Date')==content[0]).to_struct()[0]
    except Exception as e:
        print(e)
        return {}
    

In [76]:
get_information_from_url(url)

{'Date': '15/06/2025',
 'Inprogress': 'Nghiên cứu finetuning_model\n- LoRA\n- QLoRA\n',
 'Blocker': None,
 'Completed': '- Time optimization for analyis intent'}